### Use QuickSelect to find Kth largest (streaming data)

In [48]:
def FindKthLargest_QuickSelect(A, K):
    # Kth largest : index N-K
    search_idx = len(A)-K
    QuickSelect(A, 0, len(A)-1, search_idx)
    return A[search_idx]

In [39]:
def QuickSelect(A, start_idx, end_idx, search_idx):
    if len(A) <= 1:
        return
    
    pivotIndex = PickPivotIndex(A, start_idx, end_idx)
    pivotFinalIndex = Partition(A, start_idx, end_idx, pivotIndex)
    
    if pivotFinalIndex == search_idx:
        return
    elif search_idx < pivotFinalIndex:
        QuickSelect(A, start_idx, pivotFinalIndex-1, search_idx)
    else:
        QuickSelect(A, pivotFinalIndex+1, end_idx, search_idx)

In [5]:
def PickPivotIndex(A, start_idx, end_idx):
    return (start_idx + end_idx)//2

def Partition(A, start_idx, end_idx, pivotIndex):
    A[start_idx], A[pivotIndex] = A[pivotIndex], A[start_idx]
    pointer = start_idx + 1
    LeftMostBiggest = pointer
    for i in range(pointer, end_idx+1):
        if A[i] <= A[start_idx]:
            A[i], A[LeftMostBiggest] = A[LeftMostBiggest], A[i]
            LeftMostBiggest += 1
            
    pivotFinalIndex = LeftMostBiggest - 1
    A[start_idx], A[pivotFinalIndex] = A[pivotFinalIndex], A[start_idx]
    
    return pivotFinalIndex

In [30]:
def QuickSort(A, start_idx, end_idx):
    if start_idx >= end_idx:
        return
    
    pivotIndex = PickPivotIndex(A, start_idx, end_idx)
    pivotFinalIndex = Partition(A, start_idx, end_idx, pivotIndex)
    
    QuickSort(A, start_idx, pivotFinalIndex-1)
    QuickSort(A, pivotFinalIndex+1, end_idx)
    
    return A

In [34]:
QuickSort(A=[4,3,1,100,2,5], start_idx=0, end_idx=5)

[1, 2, 3, 4, 5, 100]

In [49]:
A=[4,3,1,100,2,5]
FindKthLargest_QuickSelect(A, K=5)

2

### Use Heap to find Kth largest (streaming data)

In [51]:
import heapq

In [52]:
def FindKthLargest_Heap(A, K):
    h = []
    
    for num in A:
        if len(h) < K:
            heapq.heappush(h, num)
        else:
            if h[0] < num:
                heapq.heappushpop(h, num)
                
    return h[0]

In [55]:
FindKthLargest_Heap(A=[4,3,1,100,2,5], K=4)

3

### Use Two Heaps to find Median (streaming data)

In [77]:
def returnMedian(max_heap_left, min_heap_right, new_element):
    
    # Init
    if len(max_heap_left) == 0 and len(min_heap_right) == 0:
        heapq.heappush(min_heap_right, new_element)
        return max_heap_left, min_heap_right, min_heap_right[0]
    
    # compare and put into one of the two heaps
    if new_element >= min_heap_right[0]:
        heapq.heappush(min_heap_right, new_element)
    else:
        heapq.heappush(max_heap_left, -new_element)
        
    # balance the two heaps
    if len(min_heap_right) - len(max_heap_left) > 1:
        move_element = heapq.heappop(min_heap_right)
        heapq.heappush(max_heap_left, -move_element)
    elif len(max_heap_left) - len(min_heap_right) > 1:
        move_element = -heapq.heappop(max_heap_left)
        heapq.heappush(min_heap_right, move_element)
        
    # return median
    right_root = min_heap_right[0]
    left_root = -max_heap_left[0] if len(max_heap_left) else 0
    
    if len(min_heap_right) == len(max_heap_left):
        median = (right_root + left_root)/2.0
    elif len(min_heap_right) > len(max_heap_left):
        median = right_root
    else:
        median = left_root
    
    return max_heap_left, min_heap_right, median
    

In [83]:
max_heap_left, min_heap_right = [], []

for num in [1,10,3,4,20,13,2]:
    max_heap_left, min_heap_right, median = returnMedian(
        max_heap_left, min_heap_right, new_element=num)
    print(max_heap_left, min_heap_right, median)

[] [1] 1
[-1] [10] 5.5
[-3, -1] [10] 3
[-3, -1] [4, 10] 3.5
[-3, -1] [4, 10, 20] 4
[-4, -1, -3] [10, 13, 20] 7.0
[-4, -2, -3, -1] [10, 13, 20] 4


## Heap Implementation

In [86]:
import math

int(math.log2(2 + 1)) + 1

2

In [91]:
def get_level(idx):
    return int(math.log2(idx + 1))

2**(get_level(2)) - 2

0

In [98]:
(16) // 2 - 1

7

In [181]:
class myHeap:
    
    def __init__(self, init_vals=None):
        self.data = []
        if init_vals:
            self.data = self.heapify(init_vals)
    
    @staticmethod
    def get_left_child_idx(idx):
        return idx*2 + 1
    
    @staticmethod
    def get_right_child_idx(idx):
        return idx*2 + 2
    
    @staticmethod
    def get_right_child_val(array, idx):
        right_child_idx = myHeap.get_right_child_idx(idx)
        if right_child_idx >= len(array):
            return 1e8
        else:
            return array[right_child_idx]
    
    @staticmethod
    def get_left_child_val(array, idx):
        left_child_idx = myHeap.get_left_child_idx(idx)
        if left_child_idx >= len(array):
            return 1e8
        else:
            return array[left_child_idx]
        
    @staticmethod
    def get_parent_idx(array, idx):
        return max(0, (idx + 1) // 2 - 1)
    
    @staticmethod
    def get_level(idx):
        return int(math.log2(idx + 1))
    
    @staticmethod
    def heapify(array):
        # O(N)
        
        # work on nodes from pre-terminal level up to root
        last_idx = len(array) - 1
        last_preterminal_idx = 2**(myHeap.get_level(last_idx)) - 2
        
        # heapify down for each node
        for node_idx in reversed(range(last_preterminal_idx+1)):
            myHeap.heapify_down(array, node_idx)
        
        return array
    
    @staticmethod
    def heapify_up(array, idx):
        # O(log(N))
        curr_idx = idx
        parent_idx = myHeap.get_parent_idx(array, curr_idx)
        while array[parent_idx] > array[curr_idx]:
            array[parent_idx], array[curr_idx] = \
                array[curr_idx], array[parent_idx]
            curr_idx = parent_idx
            parent_idx = myHeap.get_parent_idx(array, curr_idx)
    
    @staticmethod
    def heapify_down(array, idx):
        # O(log(N))
        curr_idx = idx
        while min(myHeap.get_left_child_val(array, curr_idx), myHeap.get_right_child_val(array, curr_idx)) < array[curr_idx]:
            if myHeap.get_left_child_val(array, curr_idx) < myHeap.get_right_child_val(array, curr_idx):
                min_child_idx = myHeap.get_left_child_idx(curr_idx)
            else:
                min_child_idx = myHeap.get_right_child_idx(curr_idx)
            try:
                array[curr_idx], array[min_child_idx] = array[min_child_idx], array[curr_idx]
            except:
                breakpoint()
            curr_idx = min_child_idx
    
    @staticmethod
    def look_root(array):
        return array[0]
    
    @staticmethod
    def pop(array):
        if len(array) == 0:
            return
        elif len(array) == 1:
            last_e = array.pop()
            return last_e
        else:
            last_e = array.pop()
            root = array[0]
            array[0] = last_e
            # heapify down at the root
            myHeap.heapify_down(array=array, idx=0)
            return root
        
    @staticmethod
    def push(array, e):
        array.append(e)
        # heapify up at the end of array
        myHeap.heapify_up(array=array, idx=len(array)-1)


In [182]:
h = myHeap([1,4,56,2,0,100, -100, -500])

In [183]:
h.data

[-500, 0, -100, 2, 1, 100, 56, 4]

In [185]:
myHeap

__main__.myHeap

In [186]:
h.data

[-500, 0, -100, 2, 1, 100, 56, 4]

In [191]:
A = [-500, 0, -100, 2, 1, 100, 56, 4]
myHeap.push(A, -50)
A

[-500, 0, -100, 2, 1, 100, 56, 4, -50] 8 3


[-500, -50, -100, 0, 1, 100, 56, 4, 2]

In [177]:
h.data

[-100, -50, 4, 0, 1, 100, 56, 2]

In [ ]:
   -100 
-50     4 
0 1  100 56 
2